In [245]:
import bqplot
import numpy as np
import pandas as pd
import ipywidgets as widgets

In [559]:
# input coordinates and class, function to modify board state 
def shape_grid(num_shapes, shape_function, size, inp, **interact_params):
    sc_x = bqplot.scales.LinearScale()
    sc_y = bqplot.scales.LinearScale()
    ax_x = bqplot.axes.Axis(scale=sc_x)
    ax_y = bqplot.axes.Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')
    ax_x.visible = False
    ax_y.visible = False
    lst_scatter_plts = []
    for i in range(num_shapes):
        lst_scatter_plts.append(
            bqplot.marks.Scatter(x=np.arange(size), 
                   y=np.arange(size),
                   scales={'x': sc_x, 'y': sc_y}, ))
    fig = bqplot.Figure(marks=lst_scatter_plts, axis=[ax_x,ax_y])
    def wrapped(**interact_params):
        nonlocal inp
        lst_scatter_plts = []
        output = shape_function(size, inp, **interact_params)
        marks = sorted(fig.marks, key=lambda x: (x.marker, x.colors[0]))
        output = sorted(output, key=lambda x: (x['options']['marker'], x['options']['color']))
        for old_points, dictionary in zip(marks, output):
            options = dictionary['options']
            coords = dictionary['coords']
            old_points.x = list(zip(*coords))[0]
            old_points.y =list(zip(*coords))[1]
            old_points.colors=[options['color']]
            old_points.marker=options['marker']
        inp = output
    display_widgets = widgets.interactive(wrapped, **interact_params)
    display(display_widgets)
    display(fig)

In [560]:
DARK_BLUE = '#475A77'
GOLDENROD = '#FEC62C'

In [561]:
def cute_shape(size, plotting_info, time, ratio):
    # build attribute grid where if there is no shape it is a 0, and dict of attribute where there is a shape
    attribute_grid = [[0] * size for _ in range(size)]
    for coords_and_options in plotting_info:
        coords = coords_and_options['coords']
        options = coords_and_options['options']
        for i,j in coords:
            attribute_grid[i][j] = options.copy()

    # check if the neighbor is satisfied
    def neighbors_are_similar(x,y):
        attribute = attribute_grid[x][y] 
        similar_count = different_count = 0
        for i in range(max(0, x-1), min(size, x+2)):
            for j in range(max(0, y-1), min(size, y+2)):
                if attribute_grid[i][j] != 0 and (i != x or j != y):
                    if attribute['marker'] == attribute_grid[i][j]['marker']:
                        similar_count += 1
                    else:
                        different_count += 1
        return different_count+similar_count != 0 and similar_count/(different_count+similar_count) > ratio
    
    # set the happiness colors for all shapes
    for coords_and_options in plotting_info:
        coords = coords_and_options['coords']
        options = coords_and_options['options']
        for i,j in coords:
            if not neighbors_are_similar(i, j):
                attribute_grid[i][j]['color'] = DARK_BLUE
            else:
                attribute_grid[i][j]['color'] = GOLDENROD
                
    # for visualizing the board changing
    if time !=0 and time < 1000:
        #find the unhappy shape
        unhappy_shape_coord = None
        unhappy_shape = None
        for i in range(size):
            if unhappy_shape:
                break
            for j in range(size):
                if attribute_grid[i][j] != 0 and attribute_grid[i][j]['color'] == DARK_BLUE:
                    unhappy_shape = attribute_grid[i][j]
                    unhappy_shape_coord = [i,j]
        displaced = True
        if unhappy_shape:
            while displaced:
                new_coord = [np.random.randint(0, size), np.random.randint(0, size)]
                if attribute_grid[new_coord[0]][new_coord[1]] == 0:
                    attribute_grid[unhappy_shape_coord[0]][unhappy_shape_coord[1]] = 0
                    attribute_grid[new_coord[0]][new_coord[1]] = unhappy_shape
                    displaced = False

    # change grid to output format
    merged_options_coordinates = []
    for i in range(size):
        for j in range(size):
            options = attribute_grid[i][j]
            if options != 0:
                added = False
                for dictionary in merged_options_coordinates:
                    if dictionary['options'] == options:
                        dictionary['coords'].append([i,j])
                        added = True
                        break
                if not added:
                    merged_options_coordinates.append({'options':options, 'coords':[[i,j]]})
    return merged_options_coordinates

In [562]:
inp= [{'options': {'color':'blue', 'marker':'diamond'}, 'coords':list(zip(np.random.choice(15,30), np.random.choice(10,30)))},
      {'options': {'color':'blue', 'marker':'circle'}, 'coords':list(zip(np.random.choice(15,30), np.random.choice(15,30)))}]

In [563]:
#widgets.Play(value=0, max=100)

In [569]:
shape_grid(4, cute_shape, 15, inp, time=widgets.Play(value=0, max=100), ratio=widgets.IntSlider(value=0.3, min=0, max=1))

A Jupyter Widget

A Jupyter Widget